In [1]:
from moccasin import setup_notebook

setup_notebook()

In [2]:
from moccasin.config import get_active_network

active_network = get_active_network()
print(active_network.name)


eth-forked


In [3]:
from boa.contracts.abi.abi_contract import ABIContract
from typing import Tuple
from moccasin.config import get_active_network
import boa

STARTING_ETH_BALANCE = int(1000e18)
STARTING_WETH_BALANCE = int(1e18)
STARTING_USDC_BALANCE = int(100e6)

def _add_eth_balance():
    boa.env.set_balance(boa.env.eoa, STARTING_ETH_BALANCE)
    
def _add_token_balance(usdc, weth):
    print(f"Starting balance of WETH: {weth.balanceOf(boa.env.eoa)}")
    print(f"Starting balance of USDC: {usdc.balanceOf(boa.env.eoa)}")
    weth.deposit(value = STARTING_WETH_BALANCE)
    our_address = boa.env.eoa
    with boa.env.prank(usdc.owner()):
        usdc.updateMasterMinter(our_address)
    usdc.configureMinter(
        our_address, STARTING_USDC_BALANCE
    )
    print("mint the usdc")
    usdc.mint(our_address, STARTING_USDC_BALANCE)
    print(f"Ending balance of WETH: {weth.balanceOf(boa.env.eoa)}")
    print(f"Ending balance of USDC: {usdc.balanceOf(boa.env.eoa)}")

def setup_script() -> Tuple[ABIContract,ABIContract,ABIContract,ABIContract]:
    print("Starting setup script...")
    
    #1. Give ourselves some ETH
    #2. Give ourselves some USDC and WETH
    active_network = get_active_network()
    
    usdc = active_network.manifest_named("usdc")
    weth = active_network.manifest_named("weth")
    
    if active_network.is_local_or_forked_network():
        _add_eth_balance()
        _add_token_balance(usdc, weth)
        
    
def moccasin_main():
    our_address = boa.env.eoa
    print(our_address,"this is the address")
    print("hello")
    setup_script()
    
moccasin_main()

0xf39Fd6e51aad88F6F4ce6aB8827279cffFb92266 this is the address
hello
Starting setup script...
Starting balance of WETH: 0
Starting balance of USDC: 0
mint the usdc
Ending balance of WETH: 1000000000000000000
Ending balance of USDC: 100000000


In [4]:
usdc = active_network.manifest_named("usdc")
weth = active_network.manifest_named("weth")



In [5]:
usdc.balanceOf(boa.env.eoa)
weth.balanceOf(boa.env.eoa)

1000000000000000000

In [6]:
from moccasin.config import get_or_initialize_config
 
config = get_or_initialize_config()
config.reload()

active_network = config.get_active_network()
aavev3_pool_address_provider = active_network.manifest_named("aavev3_pool_address_provider")
pool_address = aavev3_pool_address_provider.getPool()
print(pool_address)

config.reload()
active_network = config.get_active_network()
pool_contract = active_network.manifest_named("pool", address = pool_address)
print(pool_contract)


0x87870Bca3F3fD6335C3F4ce8392D69350B4fA4E2
<pool interface at 0x87870Bca3F3fD6335C3F4ce8392D69350B4fA4E2>


In [7]:
REFERRAL_CODE=0
def deposit(pool_contract, token, amount):
    allowed_amount = token.allowance(boa.env.eoa, pool_contract.address)
    if allowed_amount < amount:
        token.approve(pool_contract.address , amount)
    print(f"Depositing {token.name()} into Aave contract {pool_contract.address}")
    pool_contract.supply(token.address, amount, boa.env.eoa, REFERRAL_CODE)
    
usdc_balance = usdc.balanceOf(boa.env.eoa)
weth_balance = weth.balanceOf(boa.env.eoa)

if usdc_balance >0:
    deposit(pool_contract, usdc, usdc_balance)
    
if weth_balance > 0:
    deposit(pool_contract, weth, weth_balance)
(
totalCollateralBase,
        totalDebtBase,
        availableBorrowsBase,
        currentLiquidationThreshold,
        ltv,
        healthFactor,
    ) = pool_contract.getUserAccountData(boa.env.eoa)
print(f"""User account data:
        totalCollateralBase: {totalCollateralBase}
        totalDebtBase: {totalDebtBase}
        availableBorrowsBase: {availableBorrowsBase}
        currentLiquidationThreshold: {currentLiquidationThreshold}
        ltv: {ltv}
        healthFactor: {healthFactor}
          """)
 

Depositing USD Coin into Aave contract 0x87870Bca3F3fD6335C3F4ce8392D69350B4fA4E2
Depositing Wrapped Ether into Aave contract 0x87870Bca3F3fD6335C3F4ce8392D69350B4fA4E2
User account data:
        totalCollateralBase: 205779500000
        totalDebtBase: 0
        availableBorrowsBase: 165096892850
        currentLiquidationThreshold: 8275
        ltv: 8023
        healthFactor: 115792089237316195423570985008687907853269984665640564039457584007913129639935
          


In [8]:

config.reload()
active_network = config.get_active_network()

aave_protocol_data_provider = active_network.manifest_named("aave_protocol_data_provider")
a_tokens = aave_protocol_data_provider.getAllATokens()
print(a_tokens)

[getAllATokens(symbol='aEthWETH', tokenAddress=Address('0x4d5F47FA6A74757f35C14fD3a6Ef8E3C9BC514E8')), getAllATokens(symbol='aEthwstETH', tokenAddress=Address('0x0B925eD163218f6662a35e0f0371Ac234f9E9371')), getAllATokens(symbol='aEthWBTC', tokenAddress=Address('0x5Ee5bf7ae06D1Be5997A1A72006FE6C607eC6DE8')), getAllATokens(symbol='aEthUSDC', tokenAddress=Address('0x98C23E9d8f34FEFb1B7BD6a91B7FF122F4e16F5c')), getAllATokens(symbol='aEthDAI', tokenAddress=Address('0x018008bfb33d285247A21d44E50697654f754e63')), getAllATokens(symbol='aEthLINK', tokenAddress=Address('0x5E8C8A7243651DB1384C0dDfDbE39761E8e7E51a')), getAllATokens(symbol='aEthAAVE', tokenAddress=Address('0xA700b4eB416Be35b2911fd5Dee80678ff64fF6C9')), getAllATokens(symbol='aEthcbETH', tokenAddress=Address('0x977b6fc5dE62598B08C85AC8Cf2b745874E8b78c')), getAllATokens(symbol='aEthUSDT', tokenAddress=Address('0x23878914EFE38d27C4D67Ab83ed1b93A74D4086a')), getAllATokens(symbol='aEthrETH', tokenAddress=Address('0xCc9EE9483f662091a1de47

In [9]:
for a_token in a_tokens:
    if "WETH" in a_token[0]:
        a_weth = active_network.manifest_named("weth", address = a_token[1])
    if "USDC" in a_token[0]:
        a_usdc = active_network.manifest_named("usdc", address = a_token[1])
print(a_usdc)
print(a_weth)

<usdc interface at 0x98C23E9d8f34FEFb1B7BD6a91B7FF122F4e16F5c>
<weth interface at 0x4d5F47FA6A74757f35C14fD3a6Ef8E3C9BC514E8>


In [10]:
# get how much tehy are worth , to figure out to figure out how much our portfolio worth

a_usdc_balance = a_usdc.balanceOf(boa.env.eoa)
a_weth_balance = a_weth.balanceOf(boa.env.eoa)

a_usdc_balance_normalized = a_usdc_balance/ (1_000_000)
a_weth_balance_normalized = a_weth_balance/ (1_000_000_000_000_000_000)

print(a_usdc_balance_normalized)
print(a_weth_balance_normalized)


100.0
1.0


In [11]:
config.reload()

def get_price(feed_name: str) -> float:
    active_network = get_active_network()
    price_feed = active_network.manifest_named(feed_name)
    price = price_feed.latestAnswer()
    decimals = price_feed.decimals()
    decimals_normalized = 10 ** decimals
    return price / decimals_normalized

usdc_price = get_price("usdc_usd")
weth_price = get_price("eth_usd")

print(usdc_price)
print(weth_price)

0.99989
1957.806


In [12]:
usdc_value = a_usdc_balance_normalized * usdc_price
weth_value = a_weth_balance_normalized * weth_price
total_value = usdc_value + weth_value

target_usdc_value = 0.3
target_weth_value = 0.7

usdc_percent_allocation = usdc_value / total_value
weth_percent_allocation = weth_value / total_value

BUFFER = 0.1

needs_rebalancing = (
    abs(usdc_percent_allocation - target_usdc_value) > BUFFER or abs(weth_percent_allocation - target_weth_value) > BUFFER 
)
print(needs_rebalancing)
print(usdc_percent_allocation)
print(weth_percent_allocation)

True
0.04859036006988062
0.9514096399301194


In [13]:
a_weth.approve(pool_contract.address, a_weth.balanceOf(boa.env.eoa))
pool_contract.withdraw(weth.address, a_weth.balanceOf(boa.env.eoa), boa.env.eoa)

def print_token_balance():
    print(f"USDC balance: {usdc.balanceOf(boa.env.eoa)}")
    print(f"WETH balance: {weth.balanceOf(boa.env.eoa)}")
    print(f"aUSDC balance: {a_usdc.balanceOf(boa.env.eoa)}")
    print(f"aWETH balance: {a_weth.balanceOf(boa.env.eoa)}")
    
print_token_balance()

USDC balance: 0
WETH balance: 1000000000000000000
aUSDC balance: 100000000
aWETH balance: 0


In [14]:
usdc_data = {"balance":a_usdc_balance_normalized , "price":usdc_price , "contract":usdc}
weth_data = {"balance":a_weth_balance_normalized , "price":weth_price , "contract":weth}
target_allocations = {"usdc": 0.3, "weth": 0.7}


def calculate_rebalancing_trades(
    usdc_data: dict,  # {"balance": float, "price": float, "contract": Contract}
    weth_data: dict,  # {"balance": float, "price": float, "contract": Contract}
    target_allocations: dict[str, float],  # {"usdc": 0.3, "weth": 0.7}
) -> dict[str, dict]:
    """
    Calculate the trades needed to rebalance a portfolio of USDC and WETH.

    Args:
        usdc_data: Dict containing USDC balance, price and contract
        weth_data: Dict containing WETH balance, price and contract
        target_allocations: Dict of token symbol to target allocation (must sum to 1)

    Returns:
        Dict of token symbol to dict containing contract and trade amount:
            {"usdc": {"contract": Contract, "trade": int},
             "weth": {"contract": Contract, "trade": int}}
    """
    # Calculate current values
    usdc_value = usdc_data["balance"] * usdc_data["price"]
    weth_value = weth_data["balance"] * weth_data["price"]
    total_value = usdc_value + weth_value

    # Calculate target values
    target_usdc_value = total_value * target_allocations["usdc"]
    target_weth_value = total_value * target_allocations["weth"]

    # Calculate trades needed in USD
    usdc_trade_usd = target_usdc_value - usdc_value
    weth_trade_usd = target_weth_value - weth_value

    # Convert to token amounts
    return {
        "usdc": {
            "contract": usdc_data["contract"],
            "trade": usdc_trade_usd / usdc_data["price"],
        },
        "weth": {
            "contract": weth_data["contract"],
            "trade": weth_trade_usd / weth_data["price"],
        },
    }
trades = calculate_rebalancing_trades(usdc_data, weth_data , target_allocations)
print(trades)

weth_to_sell  = trades["weth"]["trade"]

{'usdc': {'contract': <usdc interface at 0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48>, 'trade': 517.4064147056175}, 'weth': {'contract': <weth interface at 0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2>, 'trade': -0.26424962432437127}}


In [15]:
config.reload()
active_network = config.get_active_network()

uniswap_swap_router = active_network.manifest_named("uniswap_swap_router")

amount_weth = abs(int(weth_to_sell * (10 ** 18)))
print(amount_weth)
weth.approve(uniswap_swap_router.address, amount_weth)

min_out = abs(int((trades["usdc"]["trade"] * (10**6)) * 0.90))
print("Let swap")
uniswap_swap_router.exactInputSingle(
    (
        weth.address,
        usdc.address, 
        3000,
        boa.env.eoa,
        amount_weth,
        min_out,
        0
    )
)

print_token_balance()

264249624324371264
Let swap
USDC balance: 515251281
WETH balance: 735750375675628736
aUSDC balance: 100000000
aWETH balance: 0


In [16]:
amount = usdc.balanceOf(boa.env.eoa)
deposit(pool_contract, usdc, amount)


Depositing USD Coin into Aave contract 0x87870Bca3F3fD6335C3F4ce8392D69350B4fA4E2


In [17]:

amount = weth.balanceOf(boa.env.eoa)
deposit(pool_contract, weth, amount)

print_token_balance()

a_usdc_balance= a_usdc.balanceOf(boa.env.eoa)
a_weth_balance= a_weth.balanceOf(boa.env.eoa)

print(f"ausdc balance {a_usdc_balance}")
print(f"a_weth balance {a_weth_balance}")

a_usdc_balance_normalized = a_usdc_balance / (1_000_000)
a_weth_balance_normalized = a_weth_balance / (1_000_000_000_000_000_000)

usdc_value = a_usdc_balance_normalized * usdc_price
weth_value = a_weth_balance_normalized * weth_price

print(usdc_value)
print(weth_value)
›
weth_percent_allocation = weth_value / (usdc_value + weth_value)
usdc_percent_allocation = usdc_value / (usdc_value + weth_value)

print(f"current percent allocation of USDC:{usdc_percent_allocation}")
print(f"current percent allocation of WETH:{weth_percent_allocation}")

Depositing Wrapped Ether into Aave contract 0x87870Bca3F3fD6335C3F4ce8392D69350B4fA4E2
USDC balance: 0
WETH balance: 0
aUSDC balance: 615251281
aWETH balance: 735750375675628736
ausdc balance 615251281
a_weth balance 735750375675628736
615.18360335909
1440.4565
current percent allocation of USDC:0.2992662005152691
current percent allocation of WETH:0.7007337994847308
